In [2]:
#CLASS PREDICTION ENCODER
predictor_col = 'Class'
result_col = 'Prediction_result'
accepted_chars = 'abcdefghijklmnopqrstuvwxyzöäü-'

# Define a mapping of chars to integers
char_to_int = dict((c, i) for i, c in enumerate(accepted_chars))
int_to_char = dict((i, c) for i, c in enumerate(accepted_chars))

# Removes all non accepted characters
def normalize(line):
    return [c.lower() for c in line if c.lower() in accepted_chars]
# Returns a list of n lists with n = word_vec_length
def name_encoding(name):
    # Encode input data to int, e.g. a->1, z->26
    integer_encoded = [char_to_int[char] for i, char in enumerate(name) if i < word_vec_length]
    # Start one-hot-encoding
    onehot_encoded = list()
    for value in integer_encoded:
        # create a list of n zeros, where n is equal to the number of accepted characters
        letter = [0 for _ in range(char_vec_length)]
        letter[value] = 1
        onehot_encoded.append(letter)
    # Fill up list to the max length. Lists need do have equal length to be able to convert it into an array
    for _ in range(word_vec_length - len(name)):
        onehot_encoded.append([0 for _ in range(char_vec_length)])
    return onehot_encoded
# Encode the output labels
def lable_encoding(result_series):
    labels = np.empty((0, 3))
    for i in result_series:
        if i == 'y1':
            labels = np.append(labels, [[1,0,0]], axis=0)
        elif i == 'y2':
            labels = np.append(labels, [[0,1,0]], axis=0)
        elif i == 'y3':
            labels = np.append(labels, [[0,0,1]], axis=0)
        else:
            labels = np.append(labels, [[1,1,1]], axis=0)
            print(i)
    return labels

In [ ]:
from pandas import read_csv
from datetime import datetime

# load data
def parse(x):
    return datetime.strptime(x, '%m/%d/%Y %H:%M:%S')

file_name = 'dataset'

dataset = read_csv(
    file_name,
    index_col=0,
    date_parser=parse,
    )
#dataset.drop('No', axis=1, inplace=True)

# manually specify column names
dataset.columns = ['X1', 'X2', 'X3', 'X4', 'Class']
dataset.index.name = 'date'
dataset.to_csv('Cleansing_new.csv')
print('Prepareprocessing Complete!')

dataset.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from keras.utils import multi_gpu_model
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
import math
import statistics
import tensorflow as tf
import os 

tf.compat.v1.enable_eager_execution()
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

df = pd.read_csv('Cleansing_new.csv')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# X = input features
dfX = df[[ 'X1', 'X2', 'X3', 'X4']]
dfY = df[[ 'Class']]
dfX.index.name = 'No'
dfY.index.name = 'No'
#ENCODING CLASS
dfY = lable_encoding(dfY['Class'])
dfY = pd.DataFrame(dfY)
dfY.columns =['y1', 'y2', 'y3']

df2 = pd.concat([dfX, dfY], axis=1)
df2.head()

In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
 
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 

values = df2.values
# integer encode direction???
encoder = LabelEncoder()
#values[:,3] = encoder.fit_transform(values[:,3])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# specify the number of lag hours
n_hours = 3  #lag
n_features = 2
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[ 2,3,4,7,8,9,12,13,14,-4,-5 ]], axis=1, inplace=True) 
reframed.head()

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = int(0.7*len(values))
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -3:]
test_X, test_y = test[:, :n_obs], test[:, -3:]
print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
#print(len(values))
#print(len(dfTest))
 

In [ ]:
from tensorflow.python.client import device_lib
from keras.utils import multi_gpu_model
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import multi_gpu_model
from tensorflow import keras

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices())   

In [ ]:
tf.config.set_soft_device_placement(True)
# tf.debugging.set_log_device_placement(True)

with tf.device('/device:GPU:1'):
    print('Build model...')
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(10,return_sequences=False,input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=3))
    model.add(tf.keras.layers.Activation('relu'))    
    optimizer = tf.keras.optimizers.Adam(0.0001)
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])
model.summary()
history = model.fit(train_X, train_y, batch_size= 16 , epochs=200, validation_data=(test_X, test_y), shuffle=False)

In [ ]:
#model save
model.save('model_name')
# list all data in history
print(history.history.keys())
print(model.summary())

In [ ]:
plt.plot(history.history['loss'], '-')
plt.plot(history.history['val_loss'], '-')
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train','Validate'],loc = 'best')
plt.grid(True)
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'], '-')
plt.plot(history.history['val_acc'], '-')
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.grid(True)
plt.show()


score_train = model.evaluate(train_X, train_y, verbose=1)
print('Train loss: {:.10f}'.format(score_train[0]) +'%')
print('Train accuracy: {:.2f}'.format(score_train[1]*100) +'%','\n')


score_test = model.evaluate(test_X, test_y, verbose=1)
print('Test loss: {:.10f}'.format(score_test[0]) +'%')
print('Test accuracy: {:.2f}'.format(score_test[1]*100) +'%','\n')